In [ ]:
import copy

def factorize(n):
  # факторизация на множители >= 2
  fact = []
  d = 2
  while (d * d <= n):
    if n % d == 0:
      fact.append(d)
      n = n // d
    else:
      d += 1
  if n > 1:
    fact.append(n)
  return sorted(fact)

def toNdim(a, n):
    while len(a) > n:
      c = copy.copy(a)
      a = sorted([c[0]*c[1]] + c[2::])
    return a

def vector_equal(a, b, n):
  # выравнивание длин векторов до заданной длины
  if len(a) > n:
    a = toNdim(a, n)
  if len(b) > n:
    b = toNdim(b, n)
  if len(a) < n:
    a = [1]* (n - len(a)) + a
  if len(b) < n:
    b = [1]* (n - len(b)) + b
  return a, b

In [ ]:
import tensorflow as tf
from tensorflow.python.framework import tensor_shape
from tensorflow.keras.initializers import *
from tensorflow.keras.backend import placeholder
from tensorflow.keras.layers import *
import random


def ttconv(inp, out_ch, d, window=(1, 1), strides=[1, 1], padding = 'SAME',
           initializers = GlorotUniform(), regularizers = None):
  ''' Tensor Train decomposition for convolution
      Источник: https:// github.com/ timgaripov/TensorNet-TF
      inp: input - [batch_size, width, height, in_chan]
      out_ch: number of output channels
      window: convolution window


      in_imag      - input image
      in_ch_dims   - factorization for dimension of input channel,   i = 0, ..., d-1
      out_ch_dims  -                                output
      d            - number of TT kernels
      ranks        - ranks for TT kernels
      ranks[0] = ranks[d] = 1
  '''
  if padding == 'same':
    padding = 'SAME'
  else:
    padding = 'VALID'

  in_h, in_w, in_ch = inp.get_shape().as_list()[1:]
  in_ch_dims, out_ch_dims = vector_equal(factorize(in_ch), factorize(out_ch), n)
  in_imag = tf.reshape(inp, [-1, in_h, in_w, in_ch])

  ranks = [0] * (d + 1)
  for i in range(0, d):
    ranks[i] = random.randint(2, 4)
  ranks[d] = 1

  filter_shape = [window[0], window[1], 1, ranks[0]] # first kernel
  if window[0]*window[1]*1*ranks[0] == 1:
    filters = tf.compat.v1.get_variable('filters',shape=filter_shape,
                                        initializer=Ones(),
                                        regularizer=regularizers)
  else:
    filters = tf.compat.v1.get_variable('filters',shape=filter_shape,
                                        initializer=initializers,
                                        regularizer=regularizers)
  kernels = []
  for i in range(d):
    kernels.append(initializers(shape=[out_ch_dims[i] * ranks[i + 1], ranks[i] * in_ch_dims[i]])) 

  conv = filters
  for i in range(d):            
    conv = tf.reshape(conv, [-1, ranks[i]])
    kernel = tf.transpose(kernels[i], [1, 0])
    kernel = tf.reshape(kernel, [ranks[i], -1])
    conv = tf.matmul(conv, kernel)

  conv_shape = [window[0], window[1]]
  order, in_order, out_order = [0, 1], [], []
  for i in range(d):
      conv_shape.append(in_ch_dims[i])
      in_order.append(2 + 2 * i)
      conv_shape.append(out_ch_dims[i])
      out_order.append(2 + 2 * i + 1)
  order += in_order + out_order
  conv = tf.reshape(conv, conv_shape)
  conv = tf.transpose(conv, order)
  conv = tf.reshape(conv, [window[0], window[1], in_ch, out_ch])

  return tf.nn.conv2d(in_imag, conv, [1] + strides + [1], padding=padding)

In [ ]:
import matplotlib
matplotlib.use("Agg")
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
import numpy as np
from numpy.testing import assert_allclose

In [ ]:
# Для работы в Google Colab
from google.colab import drive
drive.mount('/content/drive')

## miniGoogleNet

In [ ]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from keras import backend as K

def miniGoogleNet(width, height, depth, classes):
    def conv_module(x, K, kX, kY, A, B, stride, chanDim, padding="same"):
        # define convolution -> batch normalization -> ReLU
        if A <= 1 or B <= 1 or kX*kY == 1:
          # standart convolutional
          x = Conv2D(K, (kX, kY), strides=(stride[0], stride[1]), padding=padding)(x)
        else:
          # Tucker-2 +TT decomposition
          x = Conv2D(A, (1, 1), strides=(1, 1), padding=padding)(x)
          x = Conv2D(B, (kX, kY), strides=(1, 1), padding=padding)(x)
          x = ttconv(x, K, 2, window=(1, 1), strides=stride, padding=padding, regularizers=l2(0.01))
        x = BatchNormalization(axis=chanDim)(x)
        x = Activation("relu")(x)
        return x

    def inception_module(x, numK1x1, numK3x3, A, B, chanDim):
        # define two conv_modules, then concatenate them
        # across the channel dimension
        conv_1x1 = conv_module(x, numK1x1, 1, 1, -1, -1, [1, 1], chanDim)
        conv_3x3 = conv_module(x, numK3x3, 3, 3, A, B, [1, 1], chanDim)
        x = concatenate([conv_1x1, conv_3x3], axis=chanDim)
        return x

    def downsample_module(x, K, A, B, chanDim):
        # define the conv_module and pooling, then concatenate them
        # across the channel dimensions
        conv_3x3 = conv_module(x, K, 3, 3, A, B, [2, 2], chanDim, padding="valid")
        pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
        x = concatenate([conv_3x3, pool], axis=chanDim)
        return x

    # initialize the input shape to be "channels last" and the
    # channels dimension itself
    inputShape = (height, width, depth)
    chanDim = -1

    # if we are using "channels first", update the input shape
    # and channels dimension
    if K.image_data_format() == "channels_first":
        inputShape = (depth, height, width)
        chanDim = 1

    # define the model input and first CONV module
    inputs = Input(shape=inputShape)
    x = conv_module(inputs, 96, 3, 3, A, B, [1, 1], chanDim)

    # two Inception modules followed by a downsample module
    x = inception_module(x, 32, 32, A, B, chanDim)
    x = inception_module(x, 32, 48, A, B, chanDim)
    x = downsample_module(x, 80, A, B, chanDim)
    
    # four Inception modules followed by a downsample module
    x = inception_module(x, 112, 48, A, B, chanDim)
    x = inception_module(x, 96, 64, A, B, chanDim)
    x = inception_module(x, 80, 80, A, B, chanDim)
    x = inception_module(x, 48, 96, A, B, chanDim)
    x = downsample_module(x, 96, A, B, chanDim)

    # two Inception modules followed by global POOL and dropout
    x = inception_module(x, 176, 160, A, B, chanDim)
    x = inception_module(x, 176, 160, A, B, chanDim)
    x = AveragePooling2D((7, 7))(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(0.2)(x)
    

    # softmax classifier
    x = Flatten()(x)
    x = Dense(classes)(x)
    x = Activation("softmax")(x)

    # create the model
    model = Model(inputs, x, name="miniGoogleNet")
    # return the constructed network architecture
    return model

In [ ]:
def lr_schedule(epoch):
    lr = 0.01
    if epoch > 90:
        lr = 0.0001
    elif epoch > 50:
        lr = 0.001
    elif epoch > 40:
        lr = 0.01
    print("lr = ", lr)
    return lr

## init

In [ ]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = lr_schedule(0)
BATCH_SIZE = 128
NUM_EPOCHS = 100
A = 8
B = 8

lr =  0.01


In [ ]:
# initialize the label names for the CIFAR-10 dataset
labelNames = ["airplane", "automobile", "bird", "cat", "deer", "dog",
	"frog", "horse", "ship", "truck"]

# load the CIFAR-10 dataset
print("[INFO] loading CIFAR-10 dataset...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()

# scale the data to the range [0, 1]
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

# convert the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [ ]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=18, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	 horizontal_flip=True, fill_mode="nearest")

print('Loading model...')
model = miniGoogleNet(32, 32, 3, len(labelNames))

# initialize the optimizer and compile the model
opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / NUM_EPOCHS)
print("[INFO] training network...")
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [ ]:
# define the checkpoint
filepath = "/content/drive/My Drive/Colab/checkpoint_new.h5/"
lr_scheduler = LearningRateScheduler(lr_schedule)
# define the checkpoint
checkpoint = ModelCheckpoint(filepath, monitor='loss', mode='min', save_best_only=True, verbose=1)
callbacks_list =  [checkpoint, lr_scheduler]

In [ ]:
import time
start = time.time()
# train the network
H = model.fit_generator(
  aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
  validation_data=(testX, testY),
  steps_per_epoch=trainX.shape[0] // BATCH_SIZE,
  epochs=NUM_EPOCHS,
  verbose=1,
  callbacks=callbacks_list)

duration = time.time() - start
print("{} s to get output".format(duration))

import pickle
with open('/content/drive/My Drive/Colab/new2tt', 'wb') as file_pi:
        pickle.dump(H.history, file_pi)

In [ ]:
# evaluate the network
import time
print("[INFO] evaluating prediction...")
start = time.time()
predictions = model.predict(testX, batch_size=BATCH_SIZE)
duration = time.time() - start
print("{} s to get output".format(duration))
print(classification_report(testY.argmax(axis=1),
							predictions.argmax(axis=1), target_names=labelNames, digits=5))

In [ ]:
# determine the number of epochs and then construct the plot title
N = np.arange(0, NUM_EPOCHS)
title = "Training Loss and Accuracy on CIFAR-10)"

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title(title)
plt.xlabel("Epoch №")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig('/content/drive/My Drive/Colab/checkpoint_new.png')

In [ ]:
model.summary()